In [1]:
print("all_ok")

all_ok


In [2]:
%pip install python-dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os 
from dotenv import load_dotenv

load_dotenv()

if os.getenv("OPENAI_API_KEY"):
    print("OPENAI KEY Exists")
else:
    print("NO OPENAI KEY")

/Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/llmops_industry_ready_projects/document_portal/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
OPENAI KEY Exists


In [3]:
model = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

result = model.invoke("captial of france)")
print(result.content)

The capital of France is Paris.


In [4]:
print(len(embedding.embed_query("hello World!")))

1536


### Data Ingestion

In [5]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
import os  

In [7]:
pdf_path = os.path.join(os.getcwd(),"data","rag.pdf")
print(f"PDF PATH: {pdf_path}")

PDF PATH: /Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/llmops_industry_ready_projects/document_portal/notebook/data/rag.pdf


In [8]:
loader = DirectoryLoader(os.path.join(os.getcwd(),"data"),glob="*.pdf",loader_cls=PyPDFLoader)
doc = loader.load()

In [9]:
doc[0].page_content[:500]
doc[0].metadata
print(len(embedding.embed_documents(doc[0].page_content)))

2900


In [10]:
len(doc)

40

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                        chunk_overlap=200,
                                        length_function=len,
                                        separators=["\n\n","\n","."," ",""])

In [12]:
chunks = splitter.split_documents(doc)
len(chunks)

150

In [13]:
len_chunks = []
for chunk in chunks: 
    text_len = len(chunk.page_content)
    len_chunks.append(text_len)

print((round(sum(len_chunks)/len(chunks),2)))
    

1319.68


In [14]:
doc[0].metadata

{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-04-13T00:48:38+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-04-13T00:48:38+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': '/Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/llmops_industry_ready_projects/document_portal/notebook/data/rag.pdf',
 'total_pages': 19,
 'page': 0,
 'page_label': '1'}

In [15]:
from langchain_community.vectorstores import FAISS

In [17]:
faiss = FAISS(docstore="./faiss_index",embedding_function=embedding,
              index=None,index_to_docstore_id=None)
vector_store = faiss.from_documents(chunks,embedding=embedding)

In [18]:
result = vector_store.similarity_search("what is rag?",k=4)
for result in result:
    print(result.page_content[:100])
    print("-----"*30)

The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RA
------------------------------------------------------------------------------------------------------------------------------------------------------
sizes RAG’s significant advancement in enhancing the capa-
bilities of LLMs by integrating parameter
------------------------------------------------------------------------------------------------------------------------------------------------------
becoming prevalent, supporting both sequential processing and
integrated end-to-end training across 
------------------------------------------------------------------------------------------------------------------------------------------------------
16
Fig. 6. Summary of RAG ecosystem
initial learning curve. 3) Specialization - optimizing RAG to
be
-----------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":6}
)


In [20]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = """

Answer the question based on the context below. 
If the answer is not contained within the text, 
respond with "I don't know".

context: {context}

question: {question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return"\n\n".join([doc.page_content for doc in docs])

rag_chain = ({"context":retriever | format_docs , "question": RunnablePassthrough()
              }| prompt | model | StrOutputParser())
result = rag_chain.invoke("what is RAG?")
print(result)

RAG, or Retrieval-Augmented Generation, is a research paradigm that integrates retrieval techniques with generative language models to enhance their capabilities. It is categorized into three stages: Naive RAG, Advanced RAG, and Modular RAG. RAG models are designed to improve performance in various tasks by leveraging external knowledge bases and multimodal data, including text, images, audio, and video. The RAG framework focuses on the core components of "Retrieval," "Generation," and "Augmentation," and aims to optimize the performance of language models across diverse application scenarios.
